# 1. ABRIR, EDITAR E EXPORTAR ARQUIVO

In [1]:
#IMPORTA A BIBLIOTECA PANDAS E CARREGA O ARQUIVO
import pandas as pd
import locale
#CARREGA OS DADOS DE PRODUÇÃO DO ANO DE 2016
dado = pd.read_excel("../data/Boletim_1/Anexo_G-Producao_nacional_de_OLAC.xlsx", sheetname="71-Producao OLAC Geral")
locale.setlocale(locale.LC_ALL, 'portuguese_brazil')
pd.set_option('display.float_format', lambda x: locale.format('%.2f', x, grouping=True))

In [2]:
pd.__version__

'0.20.1'

In [3]:
#PREVIA DOS DADOS
dado.head()

,Data,Código do Produto,Descrição do Produto,Região do Produtor,UF do Produtor,Descrição da Operação,Volume-Litros
0,12/2016,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,NORDESTE,BA,PRODUÇÃO POR MISTURA,58045
1,12/2016,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUDESTE,MG,Produção por Mistura,121440
2,12/2016,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUDESTE,MG,Mistura de Produtos,620
3,12/2016,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,PR,Produção por Mistura,8804
4,12/2016,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUDESTE,RJ,Produção por Mistura,3927364


In [4]:
dado.shape

(997, 7)

In [5]:
#FILTRA AS COLUNAS DE INTERESSE
dado = dado[['Data', 'Descrição do Produto','Descrição da Operação','Região do Produtor', 'UF do Produtor', 'Volume-Litros']]

In [6]:
#RENOMEIA AS COLUNAS
dado.columns = ['data','category','operacao','region','uf','volume']

In [7]:
#PRÉVIA DO DATAFRAME MODIFICADO
dado.head()

,data,category,operacao,region,uf,volume
0,12/2016,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,PRODUÇÃO POR MISTURA,NORDESTE,BA,58045
1,12/2016,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,Produção por Mistura,SUDESTE,MG,121440
2,12/2016,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,Mistura de Produtos,SUDESTE,MG,620
3,12/2016,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,Produção por Mistura,SUL,PR,8804
4,12/2016,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,Produção por Mistura,SUDESTE,RJ,3927364


In [8]:
#CORREÇÃO DA ENTRADA DE DADOS
dado.loc[dado['operacao'] == 'PRODUÇÃO POR MISTURA'] = 'Produção por Mistura'

In [9]:
#CATEGORIAS DE OLAC
dado['category'].unique()

array(['Produção por Mistura', 'ENGRENAGENS E SISTEMAS CIRCULATÓRIOS',
       'Isolante Tipo A', 'Isolante Tipo B',
       'Óleos Lubrificantes Marítimos', 'Óleos Lubrificantes Ferroviários',
       'Ciclo Otto', 'Ciclo Diesel', 'Motores 2 tempos',
       'TRANSMISSÕES E SISTEMAS HIDRÁULICOS',
       'Outros Óleos Lubrificantes Acabados',
       'Óleos Lubrificantes para Aviação'], dtype=object)

In [10]:
#LIMPEZA DO DATAFRAME COM LINHAS QUE POSSUEM ERRO DE ENTRADA
mask = dado['category'].isin(['Produção por Mistura'])
dado = dado[~mask]

In [11]:
dado['category'].unique()

array(['ENGRENAGENS E SISTEMAS CIRCULATÓRIOS', 'Isolante Tipo A',
       'Isolante Tipo B', 'Óleos Lubrificantes Marítimos',
       'Óleos Lubrificantes Ferroviários', 'Ciclo Otto', 'Ciclo Diesel',
       'Motores 2 tempos', 'TRANSMISSÕES E SISTEMAS HIDRÁULICOS',
       'Outros Óleos Lubrificantes Acabados',
       'Óleos Lubrificantes para Aviação'], dtype=object)

In [12]:
dado[dado['category'] == 'Produção por Mistura']

,data,category,operacao,region,uf,volume


In [13]:
dado.shape

(828, 6)

In [14]:
df_producao = pd.DataFrame(columns=['category','uf','volume'])

# 1. CICLO DIESEL

In [15]:
diesel = pd.DataFrame(dado[dado['category'] == 'Ciclo Diesel'].groupby(by=['uf','operacao']).sum()['volume'])

In [16]:
diesel

volume
uf operacao                                  
AM Produção por Mistura                554780
BA Produção por Mistura                285024
CE Produção por Mistura                 97872
GO Produção por Mistura                 12210
MG Mistura de Produtos               17606913
   Produção por Mistura              43803080
PR Mistura de Produtos                 101177
   Produção por Mistura               1738365
RJ Mistura de Produtos               54185608
   Produção por Mistura             320192204
RS Produção por Mistura                 34430
SP Mistura de Produtos                4088742
   Produção por Mistura              11664180
   SAÍDA PARA PRODUÇÃO POR MISTURA      72048

In [17]:
diesel.index.levels[0].values

array(['AM', 'BA', 'CE', 'GO', 'MG', 'PR', 'RJ', 'RS', 'SP'], dtype=object)

In [18]:
count = 0
for uf in diesel.index.levels[0].values:
    select = diesel.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Ciclo Diesel')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Ciclo Diesel')
            count += 1

# 2. Ciclo Otto

In [19]:
otto = pd.DataFrame(dado[dado['category'] == 'Ciclo Otto'].groupby(by=['uf','operacao']).sum()['volume'])

In [20]:
otto

volume
uf operacao                                  
AM Produção por Mistura                469143
BA Produção por Mistura                951480
CE Produção por Mistura                178414
GO Produção por Mistura                  1297
MG Mistura de Produtos               26935722
   Produção por Mistura              69287772
PA Mistura de Produtos                    200
   Produção por Mistura                   200
PR Mistura de Produtos                 301377
   Produção por Mistura               5691286
RJ Mistura de Produtos               95026274
   Produção por Mistura             364319461
RS Produção por Mistura                 41716
SP Mistura de Produtos                2676316
   Produção por Mistura              39033557
   SAÍDA PARA PRODUÇÃO POR MISTURA      82556

In [21]:
otto.index.levels[0].values

array(['AM', 'BA', 'CE', 'GO', 'MG', 'PA', 'PR', 'RJ', 'RS', 'SP'], dtype=object)

In [22]:
for uf in otto.index.levels[0].values:
    select = otto.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Ciclo Otto')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Ciclo Otto')
            count += 1

# 3. ENGRENAGENS E SISTEMAS CIRCULATÓRIOS 

In [23]:
engrena = pd.DataFrame(dado[dado['category'] == 'ENGRENAGENS E SISTEMAS CIRCULATÓRIOS'].groupby(by=['uf','operacao']).sum()['volume'])

In [24]:
engrena

volume
uf operacao                                 
AM Produção por Mistura                17500
BA Produção por Mistura               183148
CE Produção por Mistura                42976
GO Produção por Mistura                  609
MG Mistura de Produtos                735220
   Produção por Mistura              1270227
PR Produção por Mistura                67814
RJ Mistura de Produtos               7912043
   Produção por Mistura             41481109
RS Produção por Mistura              2735476
SP Mistura de Produtos               2020602
   Produção por Mistura              9962960
   SAÍDA PARA PRODUÇÃO POR MISTURA     27470

In [25]:
engrena.index.levels[0].values

array(['AM', 'BA', 'CE', 'GO', 'MG', 'PR', 'RJ', 'RS', 'SP'], dtype=object)

In [26]:
for uf in engrena.index.levels[0].values:
    select = engrena.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','ENGRENAGENS E SISTEMAS CIRCULATÓRIOS')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','ENGRENAGENS E SISTEMAS CIRCULATÓRIOS')
            count += 1

# 4. Isolante Tipo A

In [27]:
isolA = pd.DataFrame(dado[dado['category'] == 'Isolante Tipo A'].groupby(by=['uf','operacao']).sum()['volume'])

In [28]:
isolA

volume
uf operacao                                 
MG Mistura de Produtos                 24256
   Produção por Mistura                36000
RJ Mistura de Produtos               1171687
   Produção por Mistura             10055061
SP Mistura de Produtos                450444
   Produção por Mistura               444104
   SAÍDA PARA PRODUÇÃO POR MISTURA      1465

In [29]:
isolA.index.levels[0].values

array(['MG', 'RJ', 'SP'], dtype=object)

In [30]:
for uf in isolA.index.levels[0].values:
    select = isolA.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Isolante Tipo A')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Isolante Tipo A')
            count += 1

# 5. Isolante Tipo B

In [31]:
isolB = pd.DataFrame(dado[dado['category'] == 'Isolante Tipo B'].groupby(by=['uf','operacao']).sum()['volume'])

In [32]:
isolB

volume
uf operacao                               
SP Mistura de Produtos                 262
   Produção por Mistura             730162
   SAÍDA PARA PRODUÇÃO POR MISTURA  133226

In [33]:
isolB.index.levels[0].values

array(['SP'], dtype=object)

In [34]:
for uf in isolB.index.levels[0].values:
    select = isolB.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Isolante Tipo B')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Isolante Tipo B')
            count += 1

# 6. Óleos Lubrificantes Marítimos

In [35]:
marit = pd.DataFrame(dado[dado['category'] == 'Óleos Lubrificantes Marítimos'].groupby(by=['uf','operacao']).sum()['volume'])
marit

volume
uf operacao                                 
RJ Mistura de Produtos               7008887
   Produção por Mistura             35580194
SP Mistura de Produtos               1077012
   Produção por Mistura              1726378
   SAÍDA PARA PRODUÇÃO POR MISTURA      5024

In [36]:
marit.index.levels[0].values

array(['RJ', 'SP'], dtype=object)

In [37]:
for uf in marit.index.levels[0].values:
    select = marit.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Óleos Lubrificantes Marítimos')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Óleos Lubrificantes Marítimos')
            count += 1

# 7. Óleos Lubrificantes Ferroviários

In [38]:
ferro = pd.DataFrame(dado[dado['category'] == 'Óleos Lubrificantes Ferroviários'].groupby(by=['uf','operacao']).sum()['volume'])
ferro

volume
uf operacao                      
RJ Mistura de Produtos    2220334
   Produção por Mistura  16717872

In [39]:
ferro.index.levels[0].values

array(['RJ'], dtype=object)

In [40]:
for uf in ferro.index.levels[0].values:
    select = ferro.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Óleos Lubrificantes Ferroviários')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Óleos Lubrificantes Ferroviários')
            count += 1

# 8. Motores 2 tempos

In [41]:
motor2T = pd.DataFrame(dado[dado['category'] == 'Motores 2 tempos'].groupby(by=['uf','operacao']).sum()['volume'])
motor2T

volume
uf operacao                                 
AM Produção por Mistura                14076
BA Produção por Mistura                 2028
CE Produção por Mistura                 2667
MG Mistura de Produtos                100786
   Produção por Mistura               246453
PR Produção por Mistura                98250
RJ Mistura de Produtos               1131687
   Produção por Mistura             12653318
RS Produção por Mistura                 7591
SP Mistura de Produtos                 16354
   Produção por Mistura               688278
   SAÍDA PARA PRODUÇÃO POR MISTURA        60

In [42]:
motor2T.index.levels[0].values

array(['AM', 'BA', 'CE', 'MG', 'PR', 'RJ', 'RS', 'SP'], dtype=object)

In [43]:
for uf in motor2T.index.levels[0].values:
    select = motor2T.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Motores 2 tempos')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Motores 2 tempos')
            count += 1

# 9. TRANSMISSÕES E SISTEMAS HIDRÁULICOS

In [44]:
trans = pd.DataFrame(dado[dado['category'] == 'TRANSMISSÕES E SISTEMAS HIDRÁULICOS'].groupby(by=['uf','operacao']).sum()['volume'])
trans

volume
uf operacao                                  
AM Produção por Mistura                475440
BA Produção por Mistura                507202
CE Produção por Mistura                105834
GO Produção por Mistura                  9988
MG Mistura de Produtos               21992434
   Produção por Mistura              59913838
PR Mistura de Produtos                 351103
   Produção por Mistura               4957181
RJ Mistura de Produtos               46918070
   Produção por Mistura             251662594
RS Mistura de Produtos                   8000
   Produção por Mistura                479858
SP Mistura de Produtos                4218534
   Produção por Mistura              28876343
   SAÍDA PARA PRODUÇÃO POR MISTURA      21358

In [45]:
trans.index.levels[0].values

array(['AM', 'BA', 'CE', 'GO', 'MG', 'PR', 'RJ', 'RS', 'SP'], dtype=object)

In [46]:
for uf in trans.index.levels[0].values:
    select = trans.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','TRANSMISSÕES E SISTEMAS HIDRÁULICOS')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','TRANSMISSÕES E SISTEMAS HIDRÁULICOS')
            count += 1

# 10. Outros Óleos Lubrificantes Acabados

In [47]:
outros = pd.DataFrame(dado[dado['category'] == 'Outros Óleos Lubrificantes Acabados'].groupby(by=['uf','operacao']).sum()['volume'])
outros

volume
uf operacao                                  
BA Produção por Mistura                 21704
CE Produção por Mistura                 26561
MG Mistura de Produtos                8025433
   Produção por Mistura              38862481
PR Produção por Mistura               1855532
RJ Mistura de Produtos                9845854
   Produção por Mistura              69686126
RS Produção por Mistura               1554714
SP Mistura de Produtos              170506709
   Produção por Mistura             141341997
   SAÍDA PARA PRODUÇÃO POR MISTURA    3004858

In [48]:
outros.index.levels[0].values

array(['BA', 'CE', 'MG', 'PR', 'RJ', 'RS', 'SP'], dtype=object)

In [49]:
for uf in outros.index.levels[0].values:
    select = outros.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Outros Óleos Lubrificantes Acabados')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Outros Óleos Lubrificantes Acabados')
            count += 1

# 11. Óleos Lubrificantes para Aviação

In [50]:
avi = pd.DataFrame(dado[dado['category'] == 'Óleos Lubrificantes para Aviação'].groupby(by=['uf','operacao']).sum()['volume'])
avi

,,volume
uf,operacao,
GO,Produção por Mistura,240


In [51]:
avi.index.levels[0].values

array(['GO'], dtype=object)

In [52]:
for uf in avi.index.levels[0].values:
    select = avi.loc[uf]
    if select.size > 1:
        if select.index[0] == "Mistura de Produtos":
            volume_1 = select.values[0][0]
        if select.index[1] == "Produção por Mistura":
            volume_2 = select.values[1][0]
            volume_corrigido = abs(volume_1-volume_2)
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_corrigido)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Óleos Lubrificantes para Aviação')
            count += 1
    else:
        if select.index[0] == "Produção por Mistura":
            volume_1 = select.values[0][0]
            #ADICIONA O VOLUME NO DATAFRAME
            df_producao.set_value(count,'volume',volume_1)
            #ADICIONA O UF NO DATAFRAME
            df_producao.set_value(count,'uf',uf)
            #ADICIONA O category NO DATAFRAME
            df_producao.set_value(count,'category','Óleos Lubrificantes para Aviação')
            count += 1

In [53]:
df_producao

,category,uf,volume
0,Ciclo Diesel,AM,554780
1,Ciclo Diesel,BA,285024
2,Ciclo Diesel,CE,97872
3,Ciclo Diesel,GO,12210
4,Ciclo Diesel,MG,26196167
5,Ciclo Diesel,PR,1637188
6,Ciclo Diesel,RJ,266006596
7,Ciclo Diesel,RS,34430
8,Ciclo Diesel,SP,7575438
9,Ciclo Otto,AM,469143


In [54]:
df_producao.loc[df_producao['category'] == 'ENGRENAGENS E SISTEMAS CIRCULATÓRIOS', 'category'] = 'Engrenagens e Sistemas Circulatórios'
df_producao.loc[df_producao['category'] == 'TRANSMISSÕES E SISTEMAS HIDRÁULICOS', 'category'] = 'Transmissões e Sistemas Hidráulicos'

In [55]:
#EXPORTA O ARQUIVO PARA CSV
df_producao.to_csv("../data/producao_olac_litros.csv", sep=',', index=False, encoding="utf-8")

In [56]:
#MODIFICAR A COLUNA PARA METROS CÚBICOS EM DECORRÊNCIA DO FEEDBACK RECEBIDO
df_producao.volume = df_producao.volume.apply(lambda x: x/1000)

In [57]:
locale.setlocale(locale.LC_ALL, 'portuguese_brazil') 
pd.set_option('display.float_format', lambda x: locale.format('%.2f', x, grouping=True))

In [58]:
df_producao.head(10)

,category,uf,volume
0,Ciclo Diesel,AM,"554,78"
1,Ciclo Diesel,BA,"285,02"
2,Ciclo Diesel,CE,"97,87"
3,Ciclo Diesel,GO,"12,21"
4,Ciclo Diesel,MG,"26.196,17"
5,Ciclo Diesel,PR,"1.637,19"
6,Ciclo Diesel,RJ,"266.006,60"
7,Ciclo Diesel,RS,"34,43"
8,Ciclo Diesel,SP,"7.575,44"
9,Ciclo Otto,AM,"469,14"


In [59]:
#EXPORTA NOVO ARQUIVO PARA CSV
df_producao.to_csv("../data/producao_olac_M3.csv", sep=';', index=False, decimal=',', encoding="utf-8")

In [60]:
dado = df_producao.copy()

# 2. ANÁLISE EXPLORATÓRIA

In [61]:
#CARREGA PLANILHA COM DADOS DAS UNIDADES PRODUTORAS DE ÓLEO LUBRIFICANTE ACABADO (OLAC)
produtores = pd.read_excel("../data/Boletim_1/Anexo_E.xlsx", sheetname="30-Mapa_Planta_Prod_OLAC")
produtores.head()

,CNPJ,Produtor,Município,UF
0,57941890000153,AGECOM PRODUTOS DE PETRÓLEO LTDA.,MAUA,SP
1,3439153000128,BIOLUB QUÍMICA LTDA -ME,SOROCABA,SP
2,33194978000190,CASTROL BRASIL LTDA,RIO DE JANEIRO,RJ
3,45036670000104,CHEMLUB PRODUTOS QUÍMICOS LTDA,VALINHOS,SP
4,5524572001084,CHEVRON BRASIL LUBRIFICANTES LTDA.,DUQUE DE CAXIAS,RJ


In [62]:
#NÚMERO DE PLANTAS DE PRODUÇÃO COM AUTORIZAÇÃO PARA PRODUZIR OLAC NO ANO DE 2016
len(produtores['CNPJ'].unique().tolist())

64

In [63]:
#NÚMERO DE PLANTAS DE PRODUÇÃO COM AUTORIZAÇÃO PARA PRODUZIR OLAC POR UNIDADE DA FEDERAÇÃO
produtores.groupby(['UF']).count()['CNPJ']

UF
AM     1
BA     1
MG     4
MT     1
PR     4
RJ    11
RS     1
SP    41
Name: CNPJ, dtype: int64

In [64]:
#AO TODO 12 UNIDADES DA FEDERAÇÃO POSSUEM REGISTRO DE PRODUÇÃO DE ÓLEO LUBRIFICANTE ACABADO (OLAC)
dado['uf'].unique(), len(dado['uf'].unique().tolist())

(array(['AM', 'BA', 'CE', 'GO', 'MG', 'PR', 'RJ', 'RS', 'SP', 'PA'], dtype=object),
 10)

In [65]:
#VOLUME DE PRODUÇÃO DE OLAC - EM METROS CÚBICOS - POR UNIDADE DA FEDERAÇÃO
producao_uf = dado.groupby(['uf']).sum().sort_values(by=['volume'],axis=0, ascending=False)
producao_uf

,volume
uf,
RJ,"896.927,49"
MG,"137.999,09"
SP,"107.755,09"
PR,"13.654,77"
RS,"4.845,78"
BA,"1.950,59"
AM,"1.530,94"
CE,"454,32"
GO,"24,34"


In [66]:
#VOLUME TOTAL PRODUZIDO DE OLAC NO ANO DE 2016
total_volume = producao_uf['volume'].sum()
total_volume

1165142.419

In [67]:
#TRANSFORMA OS DADOS PARA UMA REPRESENTAÇÃO PERCENTUAL DE SUAS RESPECTIVAS PRODUÇÕES COMPARADAS AO TOTAL PRODUZIDO NO BRASIL
producao_uf['volume'] = producao_uf['volume'].apply(lambda x : str(round((x / total_volume*100),3))+" %" )

In [68]:
producao_uf.sort_values(by='volume', ascending=False)

,volume
uf,
SP,9.248 %
RJ,76.98 %
MG,11.844 %
PR,1.172 %
RS,0.416 %
BA,0.167 %
AM,0.131 %
CE,0.039 %
GO,0.002 %


In [69]:
#VOLUME TOTAL - EM METROS CÚBICOS - PRODUZIDO POR CATEGORIA DE APLICAÇÃO DE OLAC
producao_categoria = dado.groupby(['category']).sum().sort_values(by=['volume'],axis=0, ascending=False)
producao_categoria

,volume
category,
Ciclo Otto,"355.034,44"
Ciclo Diesel,"302.399,71"
Transmissões e Sistemas Hidráulicos,"273.500,14"
Outros Óleos Lubrificantes Acabados,"123.300,54"
Engrenagens e Sistemas Circulatórios,"45.093,95"
Óleos Lubrificantes Marítimos,"29.220,67"
Óleos Lubrificantes Ferroviários,"14.497,54"
Motores 2 tempos,"12.463,83"
Isolante Tipo A,"8.901,46"


In [70]:
#TRANSFORMA OS DADOS PARA UMA REPRESENTAÇÃO PERCENTUAL DE SUAS RESPECTIVAS PRODUÇÕES COMPARADAS AO TOTAL PRODUZIDO NO BRASIL
producao_categoria['volume'] = producao_categoria['volume'].apply(lambda x : str(round((x / total_volume*100),3))+" %" )

In [71]:
producao_categoria

,volume
category,
Ciclo Otto,30.471 %
Ciclo Diesel,25.954 %
Transmissões e Sistemas Hidráulicos,23.474 %
Outros Óleos Lubrificantes Acabados,10.582 %
Engrenagens e Sistemas Circulatórios,3.87 %
Óleos Lubrificantes Marítimos,2.508 %
Óleos Lubrificantes Ferroviários,1.244 %
Motores 2 tempos,1.07 %
Isolante Tipo A,0.764 %


In [72]:
dado.groupby(['category', 'uf']).sum()

volume
category                             uf           
Ciclo Diesel                         AM     554,78
                                     BA     285,02
                                     CE      97,87
                                     GO      12,21
                                     MG  26.196,17
                                     PR   1.637,19
                                     RJ 266.006,60
                                     RS      34,43
                                     SP   7.575,44
Ciclo Otto                           AM     469,14
                                     BA     951,48
                                     CE     178,41
                                     GO       1,30
                                     MG  42.352,05
                                     PA       0,00
                                     PR   5.389,91
                                     RJ 269.293,19
                                     RS      41,72
                                     SP  36.357,24
Engrenagens e Sistemas Circulatórios AM      17,50
                                     BA     183,15
                                     CE      42,98
                                     GO       0,61
                                     MG     535,01
                                     PR      67,81
                                     RJ  33.569,07
                                     RS   2.735,48
                                     SP   7.942,36
Isolante Tipo A                      MG      11,74
                                     RJ   8.883,37
                                     SP       6,34
Isolante Tipo B                      SP     729,90
Motores 2 tempos                     AM      14,08
                                     BA       2,03
                                     CE       2,67
                                     MG     145,67
                                     PR      98,25
                                     RJ  11.521,63
                                     RS       7,59
                                     SP     671,92
Outros Óleos Lubrificantes Acabados  BA      21,70
                                     CE      26,56
                                     MG  30.837,05
                                     PR   1.855,53
                                     RJ  59.840,27
                                     RS   1.554,71
                                     SP  29.164,71
Transmissões e Sistemas Hidráulicos  AM     475,44
                                     BA     507,20
                                     CE     105,83
                                     GO       9,99
                                     MG  37.921,40
                                     PR   4.606,08
                                     RJ 204.744,52
                                     RS     471,86
                                     SP  24.657,81
Óleos Lubrificantes Ferroviários     RJ  14.497,54
Óleos Lubrificantes Marítimos        RJ  28.571,31
                                     SP     649,37
Óleos Lubrificantes para Aviação     GO       0,24